In [1]:
import mermaid as md
from mermaid.graph import Graph

#sequence = Graph('Sequence-diagram',"""
# %%{init: {'themeVariables': { 'fontSize': '10px' }}}%%

sequence = Graph('flowchart',"""
%%{init: {'themeVariables': {'nodeWidth': 500, 'wrap': false}}}%%
flowchart TB
    0@{ shape: diamond, label: "Do you have a structure of the modified protein with all hydrogens added?" }
    0-->|No|0A
    0-->|Yes|1
    0A@{ shape: rectangle, label: "All OpenFF protein loading methods require explicit hydrogens to be present"}
    0A-->|Ok, I've added explicit hydrogens|1
    1@{ shape: diamond, label: "Do you have a RDKit or OpenEye Molecule of your modified protein?" }
    1-->|Yes|1A
    1A@{ shape: rectangle, label: "Molecule.from_rdkit(rdmol)\n or \n Molecule.from_openeye(oemol)"}
    1-->|No|2
    2@{ shape: diamond, label: "Do you have an SDF file of the modified protein?"}
    2-->|Yes|2A
    2-->|No|3
    2A@{ shape: diamond, label: "Do you ALSO have a PDB file with residue information that you want to preserve in the final OpenFF Molecule?"}
    2A-->|Yes|2AA
    2A-->|No|2AB
    2AA@{ shape: rectangle, label: "ref_mol = Molecule.from_file('protein.sdf')\nfrom openmm.app import PDBFile\npdb = PDBFile('protein.pdb')\ntop = Topology.from_openmm(pdb.topology, unique_molecules=[ref_mol], positions=pdb.positions)"}
    2AB@{ shape: rectangle, label: "offmol = Molecule.from_file('protein.sdf') \n (then optionally) \n offmol.percieve_residues()"}
    3@{ shape: diamond, label: "Do you have a PDB file with CONECT records for the modified residue, and don't want to match up residue/atom names in the modified residue?"}
    3-->|Yes|3A
    3-->|No|4
    3A@{ shape: diamond, label: "Do you have an SDF/SMILES of the whole modified residue (including backbone atoms, capped with hydrogens)?"}
    3A-->|Yes|3AA
    3A-->|No|3AB
    
    3AA@{ shape: rectangle, label: "<pre style='white-space: pre-wrap; text-align: left; width: 800px;'>
    offmol = Molecule.from_file('mod_res.sdf')
    # or
    offmol = Molecule.from_smiles(modified_residue_smiles)
    for atom in offmol.atoms:
    &nbsp;&nbsp;&nbsp;&nbsp;atom.metadata['substructure_atom'] = True
    backbone_smarts = '[H:1][NH2X3][CH1X4][CH0X3](=[OH0X1])[OH1X2:2][H:3]'
    for atom_idx in offmol.chemical_environment_matches(backbone_smarts)[0]:
    &nbsp;&nbsp;&nbsp;&nbsp;offmol.atom(atom_idx).metadata['substructure_atom'] = False
    top = Topology.from_pdb('protein.pdb', _additional_substructures=[offmol])
    </pre>"}
    
    3AB@{ shape: diamond, label: "Do you have an SDF/SMILES of the modification before its attachment to a standard residue?"}
    3AB-->|Yes|3ABA
    3AB-->|No|3ABB

    3ABA@{ shape: rectangle, label: "<pre style='white-space: pre-wrap; text-align: left; width: 800px;'>
    mol = Molecule.from_file('covalent_ligand.sdf')

    # Mark the appropriate hydrogen as leaving
    matches = mol.chemical_environment_matches('S-C-C[H:1]')
    print(matches)
    for atom in mol.atoms:
    &nbsp;&nbsp;&nbsp;&nbsp;atom.metadata['substructure_atom'] = True
    # Mark the missing hydrogen as leaving
    mol.atom(matches[0][0]).metadata['substructure_atom'] = False
    
    top = Topology.from_pdb('modified_protein.pdb',
                        _additional_substructures=[mol])

    </pre>"}
    3ABB@{ shape: rectangle, label: ""}

    
    #4A@{ shape: rectangle, label: "Use Topology.from_pdb with the _additional_substructures or _custom_substructures named arguments"}
    #5A@{ shape: rectangle, label: "Use Topology.from_pdb with the _additional_substructures or _custom_substructures named arguments"}
    #6@{ shape: diamond, label: ""}
    #6A@{ shape: rectangle, label: ""}
    #6-->|Yes|4A
    #6-->|No|5

    """)
render = md.Mermaid(sequence)
render # !! note this only works in the notebook that rendered the html.